# Random Walks

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("9606.hn_HS_CX.edge", sep="\t", header=None)
data.drop([3, 4, 5], axis=1, inplace=True)
data.columns = ["node1", "node2", "weight"]
data.head()

In [ ]:
data.info()

In [ ]:
all_nodes = np.array(
    list(set(data["node1"].unique()).union(set(data["node2"].unique())))
)
print("Total number of nodes: {}".format(len(all_nodes)))

## Preprocessing Adjacency Matrix

In [ ]:
def create_symmetric_adjacency_matrix(df):
    """
    Create a symmetric adjacency matrix from a dataframe containing edges and weights.
    """
    df_symmetric = pd.concat(
        [df, df.rename(columns={"node1": "node2", "node2": "node1"})]
    )
    df_symmetric = df_symmetric.groupby(["node1", "node2"]).weight.mean().reset_index()

    adjacency_matrix = df_symmetric.pivot(
        index="node1", columns="node2", values="weight"
    ).fillna(0)

    matrix_np = adjacency_matrix.to_numpy()
    symmetrized_matrix_np = matrix_np + matrix_np.T - np.diag(matrix_np.diagonal())

    return symmetrized_matrix_np


def remove_self_loops_and_small_components(adj_matrix, threshold_components=4):
    """
    Remove self-loops and small disconnected components from the graph represented by the adjacency matrix.
    Also, keep track of the removed nodes.
    """
    G = nx.from_numpy_array(adj_matrix)
    removed_nodes = []

    for edge in nx.selfloop_edges(G):
        G.remove_edge(*edge)

    components = list(nx.connected_components(G))
    for component in components:
        if len(component) < threshold_components:
            removed_nodes.extend(component)
            for node in component:
                G.remove_node(node)

    cleaned_adj_matrix = nx.to_numpy_array(G)

    return cleaned_adj_matrix, set(removed_nodes)


def normalize_to_stochastic_matrix(adj_matrix):
    """
    Normalize an adjacency matrix so that each row sums to 1, creating a stochastic matrix.
    """
    matrix_np = np.array(adj_matrix)

    row_sums = matrix_np.sum(axis=1, keepdims=True)

    row_sums[row_sums == 0] = 1

    stochastic_matrix = matrix_np / row_sums

    return stochastic_matrix

In [ ]:
adj_mat = create_symmetric_adjacency_matrix(data)
cleaned_adj_mat, removed_nodes = remove_self_loops_and_small_components(adj_mat)

removed_nodes = list(removed_nodes)
all_nodes_cleaned = np.delete(all_nodes, removed_nodes)
avg_degree = np.count_nonzero(cleaned_adj_mat, axis=1).mean()

print(f"Number of removed nodes: {len(removed_nodes)}")
print(f"Number of nodes after cleaning: {len(all_nodes_cleaned)}")
print(f"Average degree: {avg_degree:.1f}")

In [ ]:
stochastic_matrix = normalize_to_stochastic_matrix(cleaned_adj_mat)

## Random Walk Without Restart

In [ ]:
def random_walk_without_restart_to_stationary_distribution(
    stochastic_matrix, start_node, threshold=1e-4, max_iterations=1000
):
    """
    Perform a random walk on a graph represented by a stochastic matrix, starting from a given node,
    until the stationary distribution is reached or the maximum number of iterations is exceeded.

    :param stochastic_matrix: The stochastic matrix representing the graph.
    :param start_node: The index of the starting node.
    :param threshold: The threshold for the difference between successive distribution vectors.
    :param max_iterations: The maximum number of iterations to perform.
    :return: The stationary distribution vector of the random walk.
    """
    num_nodes = stochastic_matrix.shape[0]

    q_k = np.zeros(num_nodes)
    q_k[start_node] = 1

    for _ in range(max_iterations):
        q_k_next = np.dot(q_k, stochastic_matrix)

        if np.linalg.norm(q_k_next - q_k) < threshold:
            break

        q_k = q_k_next

    return q_k_next


def visualize_distributions_heatmap(
    *distributions, node_labels=None, distribution_labels=None
):
    """
    Visualize multiple stationary distributions as a heatmap.

    :param distributions: Stationary distributions obtained from random walks.
    :param node_labels: Labels for the nodes (optional).
    :param distribution_labels: Labels for the distributions (optional).
    """
    data = np.vstack(distributions)

    plt.figure(figsize=(15, 5))
    ax = sns.heatmap(
        data,
        annot=False,
        cmap="viridis",
        yticklabels=distribution_labels
        if distribution_labels
        else [f"Dist {i + 1}" for i in range(len(distributions))],
        xticklabels=node_labels if node_labels else [],
    )
    ax.set_title("Comparison of Stationary Distributions from Random Walks")
    ax.set_xlabel("Nodes")
    ax.set_ylabel("Distributions")

    plt.show()

In [ ]:
stationary_distributions = []
for i in range(3):
    start_node = np.random.randint(stochastic_matrix.shape[0])
    print(f"Starting node: {start_node}")
    stationary_distribution = random_walk_without_restart_to_stationary_distribution(
        stochastic_matrix, start_node
    )
    stationary_distributions.append(stationary_distribution)
    print(f"Finished computing stationary distribution for starting node {start_node}")
    print(f"Sum of stationary distribution: {stationary_distribution.sum():.2f}")

In [ ]:
visualize_distributions_heatmap(*stationary_distributions)

As shown by the heatmap above, the Random Walks Without Restart do not preserve the local information of the starting node, effectively annihilating the usefulness of RW without restart for node embedding.

## Random Walk With Restart

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# TODO : verify random walk with restart implementation is correct
def random_walk_with_restart_to_stationary_distribution(
    stochastic_matrix,
    start_node,
    query_node,
    continue_prob,
    threshold=1e-10,
    max_iterations=1000,
    ignore_threshold=False,
):
    """
    Perform a Random Walk with Restart on a graph represented by a stochastic matrix, starting from a given node,
    until the stationary distribution is reached or the maximum number of iterations is exceeded.

    :param stochastic_matrix: The stochastic matrix representing the graph.
    :param start_node: The index of the starting node.
    :param restart_prob: The probability of restarting to the initial node.
    :param threshold: The threshold for the difference between successive distribution vectors.
    :param max_iterations: The maximum number of iterations to perform.
    :param ignore_threshold: Whether to ignore the threshold and always perform the maximum number of iterations.
    :return: The stationary distribution vector of the random walk.
    """
    num_nodes = stochastic_matrix.shape[0]

    # Initialize the distribution vector q_k and the restart vector v
    q_k = np.zeros(num_nodes)
    q_k[start_node] = 1
    v = np.zeros(num_nodes)
    v[query_node] = 1

    i = 0
    while True:
        if i >= max_iterations:
            print("Maximum number of iterations exceededed.")
            break

        q_k_next = (
            continue_prob * np.dot(q_k, stochastic_matrix) + (1 - continue_prob) * v
        )

        if np.linalg.norm(q_k_next - q_k) < threshold and not ignore_threshold:
            break

        q_k = q_k_next
        i += 1

    print(f"Number of iterations: {i}")

    return q_k_next

In [ ]:
def visualize_distributions_heatmap(
    *distributions, node_labels=None, distribution_labels=None
):
    """
    Visualize multiple stationary distributions as a stacked heatmap.

    :param distributions: Stationary distributions obtained from random walks.
    :param node_labels: Labels for the nodes (optional).
    :param distribution_labels: Labels for the distributions (optional).
    """
    data = np.vstack(distributions)

    plt.figure(figsize=(15, 5))
    ax = sns.heatmap(
        data,
        annot=False,
        cmap="viridis",
        yticklabels=distribution_labels
        if distribution_labels
        else [f"Dist {i + 1}" for i in range(len(distributions))],
        xticklabels=node_labels if node_labels else [],
    )
    ax.set_title("Comparison of Stationary Distributions from Random Walks")
    ax.set_xlabel("Nodes")
    ax.set_ylabel("Distributions")

    plt.show()


def visualize_top_n_nodes_line_plots(distributions, top_n=10, labels=None):
    """
    Visualize the top N nodes from each distribution using line plots.

    :param distributions: A list of distributions (numpy arrays).
    :param top_n: The number of top nodes to consider.
    """
    plt.figure(figsize=(12, 6))

    for i, distribution in enumerate(distributions):
        # Sort the distribution and pick the top N nodes
        top_nodes_indices = np.argsort(distribution)[-top_n:]
        top_nodes_values = distribution[top_nodes_indices]

        # Plotting
        plt.plot(
            top_nodes_indices,
            top_nodes_values,
            marker="o",
            label=f"Distribution {i+1}" if labels is None else labels[i],
        )

    plt.title(f"Top {top_n} Nodes in Each Distribution")
    plt.xlabel("Node Index")
    plt.ylabel("Probability")
    plt.legend()
    plt.show()


def visualize_difference_heatmaps(distributions):
    """
    Visualize the differences between distributions as a heatmap.

    :param distributions: A list of distributions (numpy arrays).
    """
    num_distributions = len(distributions)
    diff_matrix = np.zeros(
        (num_distributions, num_distributions, distributions[0].size)
    )

    # Compute the differences between each pair of distributions
    for i in range(num_distributions):
        for j in range(num_distributions):
            diff_matrix[i, j] = np.abs(distributions[i] - distributions[j])

    # Plotting the difference matrix as a heatmap
    plt.figure(figsize=(12, 12))
    for i in range(num_distributions):
        for j in range(num_distributions):
            plt.subplot(
                num_distributions, num_distributions, i * num_distributions + j + 1
            )
            sns.heatmap(
                np.reshape(diff_matrix[i, j], (1, -1)),
                annot=False,
                cmap="viridis",
                cbar=False,
            )
            plt.title(f"Diff between Dist {i+1} and Dist {j+1}")
            plt.yticks([])

    plt.show()


def visualize_top_n_nodes_box_plots(distributions, top_n=10):
    """
    Visualize the top N nodes from each distribution using box plots.

    :param distributions: A list of distributions (numpy arrays).
    :param top_n: The number of top nodes to consider.
    """
    plt.figure(figsize=(12, 6))

    # Prepare data for box plots
    top_nodes_data = []
    for distribution in distributions:
        top_nodes_indices = np.argsort(distribution)[-top_n:]
        top_nodes_data.append(distribution[top_nodes_indices])

    # Create box plots for the top N nodes
    plt.boxplot(top_nodes_data)
    plt.title(f"Top {top_n} Nodes Box Plot Comparison")
    plt.xlabel("Distribution")
    plt.ylabel("Probability")
    plt.xticks(
        range(1, len(distributions) + 1),
        [f"Dist {i+1}" for i in range(len(distributions))],
    )

    plt.show()


def visualize_top_n_node_ranking_scatter_plots(distributions, top_n=10, labels=None):
    """
    Visualize the top N node rankings in each distribution using scatter plots.

    :param distributions: A list of distributions (numpy arrays).
    :param top_n: The number of top nodes to consider.
    """
    plt.figure(figsize=(12, 6))

    for i, distribution in enumerate(distributions):
        # Sort the distribution and pick the top N nodes
        top_nodes_indices = np.argsort(distribution)[-top_n:]
        top_nodes_values = distribution[top_nodes_indices]

        # Plotting the top N nodes
        plt.scatter(
            top_nodes_indices, top_nodes_values, label=f"Distribution {i+1}" if labels is None else labels[i], alpha=0.7
        )

    plt.title(f"Top {top_n} Node Rankings in Each Distribution")
    plt.xlabel("Node Index")
    plt.ylabel("Probability")
    plt.legend()
    plt.show()


def visualize_distributions_with_pca(distributions):
    """
    Apply PCA to reduce the dimensionality of the distributions to 2D for visualization.

    :param distributions: A list of distributions (numpy arrays).
    """
    # Combine the distributions for PCA
    combined_distributions = np.stack(distributions)

    # Apply PCA to reduce to two dimensions
    pca = PCA(n_components=2)
    reduced_distributions = pca.fit_transform(combined_distributions)

    # Plot the reduced distributions
    plt.figure(figsize=(8, 6))
    for i, reduced_distribution in enumerate(reduced_distributions):
        plt.scatter(
            reduced_distribution[0],
            reduced_distribution[1],
            label=f"Distribution {i+1}",
            marker="o",
        )

    plt.title("PCA Reduced Distributions")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.legend()
    plt.show()

In [ ]:
node_1, node_2 = data.loc[0, "node1"], data.loc[0, "node2"]
print(f"Node 1: {node_1}")
print(f"Node 2: {node_2}")

In [ ]:
# find index of node 1 and node 2 in all nodes
node_1_idx = np.where(all_nodes_cleaned == node_1)[0][0]
node_2_idx = np.where(all_nodes_cleaned == node_2)[0][0]
print(f"Node 1 index: {node_1_idx}")
print(f"Node 2 index: {node_2_idx}")

### Varying restarting probability, fixed restarting node, fixed random starting node

In the first experiment we will fix N1 as the restart node, fix a random q0 as the starting node, and vary the probability of restarting from N1.

In [ ]:
continue_probs = [0.2, 0.5, 0.8]
stationary_distributions = []
start_node = 1022
for prob in continue_probs:
    print(f"Starting node: {start_node}")
    print(f"Restarting node: {node_2_idx}")
    print(f"Restart probability: {1 - prob}")
    stationary_distribution = random_walk_with_restart_to_stationary_distribution(
        stochastic_matrix, start_node, node_2_idx, prob, ignore_threshold=False
    )
    stationary_distributions.append(stationary_distribution)
    print(f"Finished computing stationary distribution for starting node {start_node}")
    print(f"Sum of stationary distribution: {stationary_distribution.sum():.2f}")

In [ ]:
# scatter top 10visualize_node_ranking_scatter_plots
visualize_top_n_node_ranking_scatter_plots(stationary_distributions, top_n=10, labels=[f"1 - p = {1 - prob:.1f}" for prob in continue_probs])

Most nodes obtain a uniform distribution, only N1 (restart node) has a higher value of 80% (the probability of restarting there). The other visited nodes are fairly similar.

Chaning the starting node doesn't change the stationary distribution.

It's likely that these nodes are in the neighborhood of the restarting node.

### Fixing $p = 0.2$, starting node, but varying restarting node